In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=4b642ad81f3517afd27157e337288c1c70ac83c47ca4eb48bf76faf42e1d8067
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
# creating a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("RoshanAssesment").getOrCreate()

In [6]:
#reading the Titanic dataset CSV file
df = spark.read.csv("titanic.csv", header = True)

In [16]:
#creating table for sql
df.createOrReplaceTempView("titanic_table")

In [7]:
# checking the schema and showing the data
df.printSchema()
df.show()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        

In [8]:
#A.How many people survive and how many people dead
# counting the number of survivors and deaths
survived_count = df.filter(df.Survived == 1).count()
died_count = df.filter(df.Survived == 0).count()

In [10]:
# printing the results
print("The people, those survives:", survived_count)
print("Deaths Count:", died_count)

The people, those survives: 342
Deaths Count: 549


In [ ]:
#For SQL

In [32]:
# counting the number of survivors
survivors = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 1").collect()[0][0]
print(f"The people, those survives: {survivors}")

The people, those survives: 342


In [33]:
# counting the number of deaths
deaths = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 0").collect()[0][0]
print(f"Deaths Count: {deaths}")

Deaths Count: 549


In [12]:
#B.How many people survive how does not have sibling
#count the number of survivors that does not have siblings
from pyspark.sql.functions import col
survived_without_siblings_count = df.filter((df.Survived == 1) & (col("SibSp") == 0)).count()

In [13]:
# printing the result
print("People who survives and does not have siblings:", survived_without_siblings_count)

People who survives and does not have siblings: 210


In [ ]:
#For SQL

In [34]:
survivors_without_siblings_count = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 1 AND SibSp = 0").collect()[0][0]

In [35]:
print(f"People who survives and does not have siblings: {survivors_without_siblings_count}")

People who survives and does not have siblings: 210


In [14]:
#C.How many people died Who does not have sibling
#count the number of death that does not have siblings
from pyspark.sql.functions import col
died_without_siblings_count = df.filter((df.Survived == 0) & (col("SibSp") == 0)).count()

In [15]:
# printing the result
print("People who died and does not have siblings:", died_without_siblings_count)

People who died and does not have siblings: 398


In [ ]:
#for SQL

In [36]:
died_without_siblings_count = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 0 AND SibSp = 0").collect()[0][0]

In [37]:
print(f"People who died and does not have siblings: {died_without_siblings_count}")

People who died and does not have siblings: 398


In [17]:
#D.Remove the NAN data from cabin and display updated list
# droping rows with NaN in the Cabin column
df_without_nan_cabin = df.na.drop(subset=["Cabin"])

In [18]:
# displaying the updated list
df_without_nan_cabin.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|   PC 17599| 71.2833|        C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|     113803|    53.1|       C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|  54|    0|    0|      17463| 51.8625|        E46|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female|   4|    1|    1|    PP 9549|    16.7|         G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|  58|    0|    0|     113783|   26.55|       C103|       S|
|         22|       1|     2|Beesley, Mr. Lawr...|  male|  34|  

In [29]:
#For SQL
df_filtered = spark.sql("SELECT * FROM titanic_table WHERE cabin IS NOT NULL")

In [30]:
# displaying the updated list
df_filtered.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|   PC 17599| 71.2833|        C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|     113803|    53.1|       C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|  54|    0|    0|      17463| 51.8625|        E46|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female|   4|    1|    1|    PP 9549|    16.7|         G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|  58|    0|    0|     113783|   26.55|       C103|       S|
|         22|       1|     2|Beesley, Mr. Lawr...|  male|  34|  

In [21]:
#E.How many male who died and how many female died?
#For Male
from pyspark.sql.functions import count
male_deaths = df.filter((df['Survived'] == 0) & (df['Sex'] == 'male')).agg(count('*')).collect()[0][0]

In [22]:
# printing the results
print("Male deaths:", male_deaths)

Male deaths: 468


In [25]:
#For Female
from pyspark.sql.functions import count
female_deaths = df.filter((df['Survived'] == 0) & (df['Sex'] == 'female')).agg(count('*')).collect()[0][0]

In [26]:
print("Female deaths:", female_deaths)

Female deaths: 81


In [27]:
#For SQL
# For Male Deaths
male_deaths = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 0 AND Sex = 'male'").collect()[0][0]
print(f"The number of male deaths: {male_deaths}")


The number of male deaths: 468


In [28]:
# For Female Deaths
female_deaths = spark.sql("SELECT COUNT(*) FROM titanic_table WHERE Survived = 0 AND Sex = 'female'").collect()[0][0]
print(f"The number of female deaths: {female_deaths}") 

The number of female deaths: 81
